Mice Movement Tracking System in Linear Discriminant Analysis.

In [ ]:
! wget https://data.caltech.edu/records/s0vdx-0k302/files/task1_classic_classification.zip?download=1
! mv task1_classic_classification.zip?download=1 task1_classic_classification.zip
! unzip task1_classic_classification.zip
! wget https://data.caltech.edu/records/s0vdx-0k302/files/calms21_convert_to_npy.py?download=1
! mv calms21_convert_to_npy.py?download=1 calms21_convert_to_npy.py
! mkdir data
! python calms21_convert_to_npy.py--input_directorytask1_classic_classification/--output_directory data/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
! pip install tensorflow

In [ ]:
data = np.load('data/calms21_task1_test.npy', allow_pickle=True)
data_dict = data.item()['annotator-id_0']
print('test data length: ', len(list(data_dict.keys())), ' videos')
data_tr = np.load('data/calms21_task1_train.npy', allow_pickle=True)
data_dict_tr = data_tr.item()['annotator-id_0']
print('train data length: ', len(list(data_dict_tr.keys())), ' videos')

In [ ]:
print(data_dict[list(data_dict.keys())[0]]['keypoints'].shape)
print('(`# frames`) x (`mouse ID`) x (`x, y coordinate`) x (`body part`)')

In [ ]:
 for key in list(data_dict.keys()):
 mouse071 = data_dict[key]
 if len(np.unique(mouse071['annotations'])) ==4:
 break
 print(mouse071['keypoints'].shape) # (`# frames`) x (`mouse ID`) x (`x, y coordinate`) x (`body part`)
 print(mouse071['scores'].shape) # (`# frames`) x (`mouse ID`) x (`body part`)
 print(mouse071['annotations'], mouse071['metadata'])
 # 'keypoints'- tracked locations of body parts on the two interacting mice
 # 'scores'- confidence estimates for the tracked keypoints
 # 'annotations'- behaviors id

In [ ]:
np.unique(mouse071['annotations'])

In [ ]:
# 1st mouse, id= 0
 body_parts_keypoints = [mouse071['keypoints'][:, :, 0, i] for i in range(7)]
 body_parts_keypoints_coords = dict()
 for i in range(7):
 body_parts_keypoints_coords[str(i) + '_x'] = body_parts_keypoints[i][:,0]
 body_parts_keypoints_coords[str(i) + '_y'] = body_parts_keypoints[i][:,1]
 body_parts_keypoints_coords[str(i) + '_score'] = mouse071['scores'][:, 0, i]
 df_mouse071_0 = pd.DataFrame(data=body_parts_keypoints_coords)
 df_mouse071_0['annotations'] = mouse071['annotations']
 df_mouse071_0.head()

In [ ]:
# 2nd mouse, id= 1
 body_parts_keypoints = [mouse071['keypoints'][:, :, 1, i] for i in range(7)]
 body_parts_keypoints_coords = dict()
 for i in range(7):
 body_parts_keypoints_coords[str(i) + '_x'] = body_parts_keypoints[i][:,0]
 body_parts_keypoints_coords[str(i) + '_y'] = body_parts_keypoints[i][:,1]
 body_parts_keypoints_coords[str(i) + '_score'] = mouse071['scores'][:, 1, i]
 df_mouse071_1 = pd.DataFrame(data=body_parts_keypoints_coords)
 df_mouse071_1['annotations'] = mouse071['annotations']
 df_mouse071_1.head()

In [ ]:
head_data = df_mouse071_0[['0_x', '0_y']]

In [ ]:
 import matplotlib.pyplot as plt
 # Extract head data
 head_data = df_mouse071_0[['0_x', '0_y']]
 # Plot head data
 plt.figure(figsize=(8, 6))
 plt.scatter(head_data['0_x'], head_data['0_y'], s=10, c='b', marker='o')
 plt.title('Head Coordinates visualization')
 plt.xlabel('x-coordinate')
 plt.ylabel('y-coordinate')
 plt.grid(True)
 plt.show()

In [ ]:
 from sklearn.model_selection import train_test_split
 from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 # Assuming df_mouse071_0 contains your DataFrame
 # Assuming the target variable is 'annotations'
 X = df_mouse071_0[['0_x', '0_y']] # Features

In [ ]:
y = df_mouse071_0['annotations'] # Target variable
 # Split the data into training and testing sets
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,␣
 ↪random_state=42)
 # Initialize and fit the LDA model
 lda = LinearDiscriminantAnalysis()
 lda.fit(X_train, y_train)
 # Evaluate the model
 train_accuracy = lda.score(X_train, y_train)
 test_accuracy = lda.score(X_test, y_test)
 print("Training accuracy:", train_accuracy)
 print("Testing accuracy:", test_accuracy)

In [ ]:
 from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 import matplotlib.pyplot as plt
 # Assuming df_mouse071_0 contains your DataFrame
 X = df_mouse071_0[['0_x', '0_y']] # Features
 y = df_mouse071_0['annotations'] # Target variable
 # Split the data into training and testing sets
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,␣
 ↪random_state=42)
 plt.figure(figsize=(14,5))
 # Plot using svd solver
 plt.subplot(1,3,1)
 lda = LinearDiscriminantAnalysis(solver='svd')
 lda.fit(X_train, y_train)
 y_predict = lda.predict(X_test)
 plt.scatter(X_test['0_x'], y_test, s=10, c='b', marker="s", label='testing')
 plt.scatter(X_test['0_x'], y_predict, s=10, c='r', marker="o", label='predict')
 plt.legend()
 score = lda.score(X_test, y_test)
 plt.title('Accuracy of the svd solver is '+str(score))
 # Plot using lsqr solver
 plt.subplot(1,3,2)
 lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

In [ ]:
lda.fit(X_train, y_train)
 y_predict = lda.predict(X_test)
 plt.scatter(X_test['0_x'], y_test, s=10, c='b', marker="s", label='testing')
 plt.scatter(X_test['0_x'], y_predict, s=10, c='r', marker="o", label='predict')
 plt.legend()
 score = lda.score(X_test, y_test)
 plt.title('Accuracy of the lsqr solver is '+str(score))
 # Plot using eigen solver
 plt.subplot(1,3,3)
 lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
 lda.fit(X_train, y_train)
 y_predict = lda.predict(X_test)
 plt.scatter(X_test['0_x'], y_test, s=10, c='b', marker="s", label='testing')
 plt.scatter(X_test['0_x'], y_predict, s=10, c='r', marker="o", label='predict')
 plt.legend()
 score = lda.score(X_test, y_test)
 plt.title('Accuracy of the eigen solver is '+str(score))
 plt.tight_layout()
 plt.show()

In [ ]:
 import numpy as np
 import matplotlib.pyplot as plt
 from sklearn.model_selection import train_test_split
 from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 # Assuming head data is stored in X (features) and y (target variable)
 X = head_data[['0_x', '0_y']]
 y = df_mouse071_0['annotations']
 # Define range of testing sizes
 test_sizes = np.arange(0.05, 0.95, 0.05)
 accuracies = []
 # Iterate over different testing sizes
 for test_size in test_sizes:
 # Split the data into training and testing sets
 X_train, X_test, y_train, y_test = train_test_split(X, y,␣
 ↪test_size=test_size, random_state=42)
 # Initialize and fit the LDA model
 lda = LinearDiscriminantAnalysis()
 lda.fit(X_train, y_train)
 # Evaluate the model
 accuracy = lda.score(X_test, y_test)
 accuracies.append(accuracy)
 # Find the optimal test size with maximum accuracy
 optimal_test_size = test_sizes[np.argmax(accuracies)]
 max_accuracy = np.max(accuracies)
 print("Optimal Test Size:", optimal_test_size)
 print("Maximum Accuracy:", max_accuracy)
 # Plot accuracy vs. test size
 plt.plot(test_sizes, accuracies, marker='o')
 plt.title('Accuracy vs. Test Size')
 plt.xlabel('Test Size')
 plt.ylabel('Accuracy')
 plt.grid(True)
 plt.show()

Neck Data

In [ ]:
import matplotlib.pyplot as plt
 # Extract neck data
 neck_data = df_mouse071_0[['1_x', '1_y']]
 # Plot neck data
 plt.figure(figsize=(8, 6))
 plt.scatter(neck_data['1_x'], neck_data['1_y'], s=10, c='r', marker='o')
 plt.title('Neck Coordinates visualization')
 plt.xlabel('x-coordinate')
 plt.ylabel('y-coordinate')
 plt.grid(True)
 plt.show()

In [ ]:
 from sklearn.model_selection import train_test_split
 from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 # Assuming df_mouse071_0 contains your DataFrame
 # Assuming the target variable is 'annotations'
 X = df_mouse071_0[['1_x', '1_y']] # Features
 y = df_mouse071_0['annotations'] # Target variable
 # Split the data into training and testing sets
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,␣
 ↪random_state=42)
 # Initialize and fit the LDA model
 lda = LinearDiscriminantAnalysis()
 lda.fit(X_train, y_train)
 # Evaluate the model
 train_accuracy = lda.score(X_train, y_train)
 test_accuracy = lda.score(X_test, y_test)
 print("Training accuracy:", train_accuracy)
 print("Testing accuracy:", test_accuracy)

In [ ]:
 from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 import matplotlib.pyplot as plt
 # Assuming df_mouse071_0 contains your DataFrame
 X = df_mouse071_0[['1_x', '1_y']] # Features
 y = df_mouse071_0['annotations'] # Target variable
 # Split the data into training and testing sets
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,␣
 ↪random_state=42)
 plt.figure(figsize=(14,5))
 # Plot using svd solver
 plt.subplot(1,3,1)
 lda = LinearDiscriminantAnalysis(solver='svd')
 lda.fit(X_train, y_train)
 y_predict = lda.predict(X_test)
 plt.scatter(X_test['1_x'], y_test, s=10, c='b', marker="s", label='testing')
 plt.scatter(X_test['1_x'], y_predict, s=10, c='r', marker="o", label='predict')
 plt.legend()
 score = lda.score(X_test, y_test)
 plt.title('Accuracy of the svd solver is '+str(score))
 # Plot using lsqr solver
 plt.subplot(1,3,2)
 lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
 lda.fit(X_train, y_train)
 y_predict = lda.predict(X_test)
 plt.scatter(X_test['1_x'], y_test, s=10, c='b', marker="s", label='testing')
 plt.scatter(X_test['1_x'], y_predict, s=10, c='r', marker="o", label='predict')
 plt.legend()
 score = lda.score(X_test, y_test)
 plt.title('Accuracy of the lsqr solver is '+str(score))
 # Plot using eigen solver
 plt.subplot(1,3,3)
 lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
 lda.fit(X_train, y_train)
 y_predict = lda.predict(X_test)
 plt.scatter(X_test['1_x'], y_test, s=10, c='b', marker="s", label='testing')
 plt.scatter(X_test['1_x'], y_predict, s=10, c='r', marker="o", label='predict')
 plt.legend()
 score = lda.score(X_test, y_test)
 plt.title('Accuracy of the eigen solver is '+str(score))
 plt.tight_layout()
 plt.show()

In [ ]:
 # Assuming df_mouse071_0 contains your DataFrame
 names = df_mouse071_0.columns.to_numpy()
 name_at_index_21 = names[21]
 print(name_at_index_21)
 column_names_array = df_mouse071_0.columns.to_numpy()
 print(column_names_array)

Function of getting all the body parts and getting all the graphs at the same time

In [ ]:
 import matplotlib.pyplot as plt
 # Plot all data points
 plt.figure(figsize=(8, 6))
 for i in range(7): # Assuming you have 7 sets of coordinates
 x_col = f'{i}_x'
 y_col = f'{i}_y'
 plt.scatter(df_mouse071_0[x_col], df_mouse071_0[y_col], s=10, label=f'Point␣{i}')
 plt.title('All Body Parts Coordinates visualization')
 plt.xlabel('x-coordinate')
 plt.ylabel('y-coordinate')
 plt.legend()
 plt.grid(True)
 plt.show()


In [ ]:
print("Data type of y_train:", y_train.dtype)
 print("Data type of y_test:", y_test.dtype)

Optimizing accuracy improvement for body parts using LDA model for
adding feature  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
# Function to optimize accuracy for a given body part
def optimize_accuracy(df, body_part):
# Extract body part data and time data
body_part_data = df[[f'{body_part}_x', f'{body_part}_y']]
time_data = df['time']  # Assuming 'time' is the column name for time
feature
# Combine body part data and time data
X = pd.concat([body_part_data, time_data], axis=1)
y = df['annotations']
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.3, random_state=42)
    # Initialize and fit the LDA model
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    # Calculate accuracy before optimization
    accuracy_before_optimization = lda.score(X_test, y_test)
    # Initialize variables to store optimized parameters and accuracy
    best_test_size = 0.3
    best_accuracy = accuracy_before_optimization
    accuracy = []
    test_sizes = []
    # Perform optimization by changing the test size
    for test_size in np.arange(0.1, 0.9, 0.1):
        # Split data with the optimized test size
        X_train_optimized, X_test_optimized, y_train_optimized,
y_test_optimized = train_test_split(X, y, test_size=test_size,
random_state=42)
        # Fit the model with the optimized data
        lda.fit(X_train_optimized, y_train_optimized)
        # Calculate accuracy after optimization
        accuracy_after_optimization = lda.score(X_test_optimized,
y_test_optimized)
        accuracy.append(accuracy_after_optimization)
        test_sizes.append(test_size)
        # Update best accuracy and test size if improvement is found
        if accuracy_after_optimization > best_accuracy:
            best_accuracy = accuracy_after_optimization
            best_test_size = test_size
    # Calculate accuracy improvement
    accuracy_improvement = best_accuracy - accuracy_before_optimization
    # Perform K-Means clustering
    # Extract body part data and time data
    body_part_data = df[[f'{body_part}_x', f'{body_part}_y']]
    time_data = df['time']  # Assuming 'time' is the column name for time
feature
    # Combine body part data and time data
    X_cluster = pd.concat([body_part_data, time_data], axis=1)
    # Standardize the data
    scaler = StandardScaler()
    body_part_data_scaled = scaler.fit_transform(X_cluster)
    # Initialize the K-Means clustering model
    kmeans = KMeans(n_clusters=2, random_state=42)  # Assuming 2 clusters
for each body part
    # Fit the model to the data
    kmeans.fit(body_part_data_scaled)
    # Get the cluster labels for each data point
    labels = kmeans.labels_
    # Add the cluster labels to the original DataFrame
    df[f'{body_part}_cluster'] = labels
    return accuracy_improvement, best_test_size
# Assuming df_mouse071_0 contains the DataFrame for mouse 071
# Define the list of body parts
body_parts = ['0', '1', '2', '3', '4', '5', '6']
# Initialize a dictionary to store accuracy improvements and best test
sizes for each body part
accuracy_improvements = {}
best_test_sizes = {}
# Iterate over each body part and optimize accuracy
for body_part in body_parts:
    accuracy_improvement, best_test_size = optimize_accuracy(df_mouse071_0,
body_part)
    accuracy_improvements[body_part] = accuracy_improvement
    best_test_sizes[body_part] = best_test_size
# Plot the accuracy improvements as a bar plot
plt.figure(figsize=(10, 6))
plt.bar(accuracy_improvements.keys(), accuracy_improvements.values())
plt.xlabel('Body Part')
plt.ylabel('Accuracy Improvement')
plt.title('Accuracy Improvement after adding feature')
plt.show()
# Print the best test sizes for each body part
print("Best Test Sizes:")
for body_part in body_parts:
    print(f"Body Part {body_part}: {best_test_sizes[body_part]}")

Optimizing accuracy improvement for body parts using LDA model for Adding Time
Feature and Changing Test Size

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Function to optimize accuracy for a given body part
def optimize_accuracy(df, body_part):
    # Extract body part data and time data
    body_part_data = df[[f'{body_part}_x', f'{body_part}_y', 'time']]

    # Split the data into features (X) and target variable (y)
    X = body_part_data
    y = df['annotations']

    # Initialize and fit the LDA model
    lda = LinearDiscriminantAnalysis()

    # Calculate accuracy for default test size
    X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.3, random_state=42)
    lda.fit(X_train, y_train)
    accuracy_default = lda.score(X_test, y_test)

    # Initialize variables to store best accuracy and parameters
    best_accuracy = accuracy_default
    best_test_size = 0.3
    best_time_added = False

    # Try different test sizes and check accuracy improvement after
adding time feature
    for test_size in np.arange(0.1, 0.9, 0.1):
        X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=test_size, random_state=42)
        lda.fit(X_train, y_train)
        accuracy_test_size = lda.score(X_test, y_test)

        # Add time feature
        X_train['time'] = np.random.rand(X_train.shape[0])  #
Placeholder for actual time data
        X_test['time'] = np.random.rand(X_test.shape[0])  # Placeholder
for actual time data

        lda.fit(X_train, y_train)
        accuracy_with_time = lda.score(X_test, y_test)

        # Check if accuracy improved
        if accuracy_with_time > best_accuracy:
            best_accuracy = accuracy_with_time
            best_test_size = test_size
            best_time_added = True

    accuracy_improvement = best_accuracy - accuracy_default

    return accuracy_improvement, best_test_size, best_time_added

# Assuming df_mouse071_0 contains the DataFrame for mouse 071
# Define the list of body parts
body_parts = ['0', '1', '2', '3', '4', '5', '6']

# Initialize dictionaries to store accuracy improvements, best test
sizes, and time added
accuracy_improvements = {}
best_test_sizes = {}
time_added = {}

# Iterate over each body part and optimize accuracy
for body_part in body_parts:
    accuracy_improvement, best_test_size, time_added_flag =
optimize_accuracy(df_mouse071_0, body_part)
    accuracy_improvements[body_part] = accuracy_improvement
    best_test_sizes[body_part] = best_test_size
    time_added[body_part] = time_added_flag

# Plot the accuracy improvements as a bar plot
plt.figure(figsize=(10, 6))
plt.bar(accuracy_improvements.keys(), accuracy_improvements.values())
plt.xlabel('Body Part')
plt.ylabel('Accuracy Improvement')
plt.title('Accuracy Improvement after Adding Time Feature and Changing
Test Size')
plt.show()

# Print the best test sizes for each body part
print("Best Test Sizes:")
for body_part in body_parts:
    print(f"Body Part {body_part}: {best_test_sizes[body_part]}")

# Print whether time feature was added for each body part
print("\nTime Feature Added:")
for body_part in body_parts:
    print(f"Body Part {body_part}: {'Yes' if time_added[body_part] else
'No'}")

Optimizing accuracy improvement for body parts using LDA model for changing solver

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Function to optimize accuracy for a given body part
def optimize_accuracy(df, body_part):
    # Extract body part data and time data
    body_part_data = df[[f'{body_part}_x', f'{body_part}_y', 'time']]

    # Split the data into features (X) and target variable (y)
    X = body_part_data
    y = df['annotations']

    # Initialize variables to store best accuracy and parameters
    best_accuracy = 0
    best_solver = None

    # Try different solvers and check accuracy improvement after adding
time feature
    solvers = ['svd', 'lsqr', 'eigen']
    for solver in solvers:
        # Initialize and fit the LDA model with the current solver
        lda = LinearDiscriminantAnalysis(solver=solver)

        # Calculate accuracy for default test size
        X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.3, random_state=42)
        lda.fit(X_train, y_train)
        accuracy_default = lda.score(X_test, y_test)

        # Add time feature
        X_train['time'] = np.random.rand(X_train.shape[0])  #
Placeholder for actual time data
        X_test['time'] = np.random.rand(X_test.shape[0])  # Placeholder
for actual time data

        lda.fit(X_train, y_train)
        accuracy_with_time = lda.score(X_test, y_test)

        # Check if accuracy improved
        accuracy_improvement = accuracy_with_time - accuracy_default
        if accuracy_improvement > best_accuracy:
            best_accuracy = accuracy_improvement
            best_solver = solver

    return best_accuracy, best_solver

# Assuming df_mouse071_0 contains the DataFrame for mouse 071
# Define the list of body parts
body_parts = ['0', '1', '2', '3', '4', '5', '6']

# Initialize dictionaries to store accuracy improvements and best
solvers
accuracy_improvements = {}
best_solvers = {}

# Iterate over each body part and optimize accuracy
for body_part in body_parts:
    accuracy_improvement, best_solver =
optimize_accuracy(df_mouse071_0, body_part)
    accuracy_improvements[body_part] = accuracy_improvement
    best_solvers[body_part] = best_solver

# Plot the accuracy improvements as a bar plot
plt.figure(figsize=(10, 6))
plt.bar(accuracy_improvements.keys(), accuracy_improvements.values())
plt.xlabel('Body Part')
plt.ylabel('Accuracy Improvement')
plt.title('Accuracy Improvement after Changing Solver')
plt.show()

# Print the best solvers for each body part
print("Best Solvers:")
for body_part in body_parts:
    print(f"Body Part {body_part}: {best_solvers[body_part]}")

Optimizing accuracy improvement for body parts using LDA model for changing solver
and changing test size

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Function to optimize accuracy for a given body part
def optimize_accuracy(df, body_part):
    # Extract body part data and time data
    body_part_data = df[[f'{body_part}_x', f'{body_part}_y', 'time']]

    # Split the data into features (X) and target variable (y)
    X = body_part_data
    y = df['annotations']

    # Initialize variables to store best accuracy and parameters
    best_accuracy = 0
    best_solver = None
    best_test_size = 0

    # Try different solvers and test sizes and check accuracy
improvement after adding time feature
    solvers = ['svd', 'lsqr', 'eigen']
    for solver in solvers:
        for test_size in np.arange(0.1, 0.9, 0.1):
            # Initialize and fit the LDA model with the current solver
            lda = LinearDiscriminantAnalysis(solver=solver)

            # Calculate accuracy for the current test size
            X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=test_size, random_state=42)
            lda.fit(X_train, y_train)
            accuracy_default = lda.score(X_test, y_test)

            # Add time feature
            X_train['time'] = np.random.rand(X_train.shape[0])  #
Placeholder for actual time data
            X_test['time'] = np.random.rand(X_test.shape[0])  #
Placeholder for actual time data

            lda.fit(X_train, y_train)
            accuracy_with_time = lda.score(X_test, y_test)

            # Check if accuracy improved
            accuracy_improvement = accuracy_with_time -
accuracy_default
            if accuracy_improvement > best_accuracy:
                best_accuracy = accuracy_improvement
                best_solver = solver
                best_test_size = test_size

    return best_accuracy, best_solver, best_test_size

# Assuming df_mouse071_0 contains the DataFrame for mouse 071
# Define the list of body parts
body_parts = ['0', '1', '2', '3', '4', '5', '6']

# Initialize dictionaries to store accuracy improvements, best solvers,
and best test sizes
accuracy_improvements = {}
best_solvers = {}
best_test_sizes = {}

# Iterate over each body part and optimize accuracy
for body_part in body_parts:
    accuracy_improvement, best_solver, best_test_size =
optimize_accuracy(df_mouse071_0, body_part)
    accuracy_improvements[body_part] = accuracy_improvement
    best_solvers[body_part] = best_solver
    best_test_sizes[body_part] = best_test_size

# Plot the accuracy improvements as a bar plot
plt.figure(figsize=(10, 6))
plt.bar(accuracy_improvements.keys(), accuracy_improvements.values())
plt.xlabel('Body Part')
plt.ylabel('Accuracy Improvement')
plt.title('Accuracy Improvement after Changing Solver and Test Size')
plt.show()

# Print the best solvers and test sizes for each body part
print("Best Solvers and Test Sizes:")
for body_part in body_parts:
    print(f"Body Part {body_part}: Solver - {best_solvers[body_part]},
Test Size - {best_test_sizes[body_part]}")